# Chapter 9  
## Metaprogramming

One of the most important mantras of software development is "don’t repeat yourself."  
That is, any time you are faced with a problem of creating highly repetitive code (or cutting and pasting source code), it often pays to look for a more elegant solution.  
In Python, such problems are often solved under the category of "metaprogramming."  
In a nutshell, metaprogramming is about creating functions and classes whose main goal is to manipulate code by modifying, generating, or wrapping existing code.  
The main features for this include decorators, class decorators, and metaclasses.  
However, a variety of other useful topics -- including signature objects, execution of code with exec(), and inspecting the internals of functions and classes -- enter the picture.  
The main purpose of this chapter is to explore various metaprogramming techniques and to give examples of how they can be used to customize the behavior of Python to your own whims.

## 9.1 Putting a Wrapper Around a Function

### Problem

You want to put a wrapper layer around a function that adds extra processing, such as logging, timing, and so forth.

### Solution

You can define a decorator function to wrap a function with extra code.

In [1]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Now let's make our decorator useful.

In [2]:
@timethis
def countdown(n):
    """
    Yes, just a countdown.
    """
    while n > 0:
        n -= 1

In [3]:
countdown(10000)

countdown 0.0007228851318359375


In [4]:
countdown(100000)

countdown 0.006545066833496094


### Discussion

A decorator is a function that accepts a function as input and returns a new function as output.  
Whenever you write code like the following:

In [5]:
@timethis
def countdown(n):
    '''do something'''

it's the same as if you had executed these separate steps:

In [6]:
def countdown(n):
    '''...'''
countdown = timethis(countdown)

Python's built-in decorators, like `@staticmethod`, `@classmethod`, and `@property` work in the same way.  
For example, the following two code fragments will give an equivalent result:

In [7]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    # Equivalent definition of a class method.
    def method(cls):
        pass
    method = classmethod(method)

The code inside a decorator typically involves creating a new function that accepts any arguments using `*args` and `**kwargs`, as shown with the `wrapper()` function in this recipe.  
Inside this function, you place a call to the original input function and return its result.  
However, you also place whatever extra code you want to add, like our timing function.  
The newly created function wrapper is returned as a result and takes the place of the original function.  
It’s critical to emphasize that decorators generally do not alter the calling signature or return value of the function being wrapped.  
The use of `*args` and `**kwargs` is there to make sure that any input arguments can be accepted.  
The return value of a decorator is almost always the result of calling `func(*args, **kwargs)`, where `func` is the original unwrapped function.  
When first learning about decorators, it is usually very easy to get started with some simple examples, such as the one shown.  
However, if you are going to write decorators for real, there are some subtle details to consider.  
For example, the use of the decorator `@wraps(func)` in the solution is an easy to forget but important technicality related to preserving function metadata, which is described in the next recipe.  
The next few recipes that follow fill in some details that will be important if you wish to write decorator functions of your own.

## 9.2. Preserving Function Metadata When Writing Decorators

### Problem

You've written a decorator, but when you apply it to a function, important metadata such as the name, docstring, annotations, and calling signature are lost.

### Solution

Whenever you define a decorator, you should always remember to apply the `@wraps` decorator from the `functools` library to the underlying wrapper function.

In [8]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

Let's use the decorator and examine the resulting function metadata:

In [9]:
@timethis
def countdown(n:int):
    """
    Countdown the integers.
    """
    while n > 0:
        n -= 1

In [10]:
countdown(10000)

countdown 0.0006909370422363281


In [11]:
countdown(100000)

countdown 0.00626826286315918


In [12]:
countdown.__name__

'countdown'

In [13]:
countdown.__doc__

'\n    Countdown the integers.\n    '

In [14]:
countdown.__annotations__

{'n': int}

### Discussion

Copying decorator metadata is an important part of writing decorators.  
If you forget to use `@wraps`, you’ll find that the decorated function loses all sorts of useful information.  
For instance, if omitted, the metadata in the last example would look like this:

An important feature of the `@wraps` decorator is that it makes the wrapped function available to you in the `__wrapped__` attribute.  
For example, if you want to access the wrapped function directly, you could do this:

The presence of the `__wrapped__` attribute also makes decorated functions properly expose the underlying signature of the wrapped function.

In [15]:
from inspect import signature
print(signature(countdown))

(n:int)


One common question that sometimes arises is how to make a decorator that directly copies the calling signature of the original function being wrapped (as opposed to using `*args` and `**kwargs`).  
In general, this is difficult to implement without resorting to some trick involving the generator of code strings and `exec()`.  
Frankly, you’re usually best off using `@wraps` and relying on the fact that the underlying function signature can be propagated by access to the underlying `__wrapped__` attribute.  
See Recipe 9.16 for more information about signatures.

## 9.3. Unwrapping a Decorator

### Problem

A decorator has been applied to a function, but you want to overrride it and gain access to the original unwrapped function.

### Solution

Assuming that the decorator has been implemented properly using `@wraps` (see Recipe 9.2), you can usually gain access to the original function by accessing the `__wrapped__` attribute.

### Discussion

Gaining direct access to the unwrapped function behind a decorator can be useful for debugging, introspection, and other operations involving functions.  
However, this recipe only works if the implementation of a decorator properly copies metadata using `@wraps` from the functools module or sets the `__wrapped__` attribute directly.  
If multiple decorators have been applied to a function, the behavior of accessing `__wrapped__` is currently undefined and should probably be avoided.  
In Python 3.3, it bypasses all of the layers.  
For example, suppose you have code like this:

In [16]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

In [17]:
@decorator1
@decorator2
def add(x, y):
    return x + y

Now let's call the decorated function and the original function through `__wrapped__`.

In [18]:
add(2, 3)

Decorator 1
Decorator 2


5

In [19]:
add.__wrapped__(2, 3)

Decorator 2


5

However, this behavior has been reported as a bug (see `http://bugs.python.org/issue17482`) and may be changed to expose the proper decorator chain in a future release.
Last, but not least, be aware that not all decorators utilize `@wraps`, and thus, they may not work as described.  
In particular, the built-in decorators `@staticmethod` and `@classmethod` create descriptor objects that don’t follow this convention (instead, they store the original function in a `__func__` attribute).  
Your mileage may vary.

## 9.4. Defining a Decorator That Takes Arguments

### Problem

You want to write a decorator function that takes arguments.

### Solution

Let’s illustrate the process of accepting arguments with an example.  
Suppose you want to write a decorator that adds logging to a function, but allows the user to specify the logging level and other details as arguments.  
Here is how you might define the decorator:

In [20]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function.
    If name and message aren't specified, they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.level(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

Test drive our code:

In [21]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

On first glance, the implementation looks tricky, but the idea is relatively simple.  
The outermost function `logged()` accepts the desired arguments and simply makes them available to the inner functions of the decorator.  
The inner function `decorate()` accepts a function and puts a wrapper around it as normal.  
The key part is that the wrapper is allowed to use the arguments passed to `logged()`.

### Discussion

Writing a decorator that takes arguments is tricky because of the underlying calling sequence involved.  
For example, if you have code like this:

The decoration process evaluates as follows:

Carefully observe that the result of `decorator(x, y, z)` must be a callable which, in turn, takes a function as input and wraps it.  
See Recipe 9.7 for another example of a decorator taking arguments.

## 9.5. Defining a Decorator with User Adjustable Attributes

### Problem

You want to write a decorator function that wraps a function, but has user adjustable attributes that can be used to control the behavior of the decorator at runtime.

### Solution

Here is a solution that expands on the last recipe by introducing accessor functions that change internal variables through the use of nonlocal variable declarations.  
The accessor functions are then attached to the wrapper function as function attributes.

In [22]:
from functools import wraps, partial
import logging

# Utility decorator to attach a function as an attribute of obj:
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func
def logged(level, name=None, message=None):
    """
    Add logging to a function.
    If name and message aren't specified, they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # Attach setter functions:
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
    return decorate

# Example use:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

Here is an interactive session that shows the various attributes being changed after definition:

In [23]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

DEBUG:__main__:add


5

In [24]:
# Change the log message:
add.set_message('Add called')
add(2, 3)

DEBUG:__main__:Add called


5

In [25]:
# Change the log level:
add.set_level(logging.WARNING)
add(2, 3)

5

### Discussion

The key to this recipe lies in the accessor functions, like `set_message()` and `set_level()`, that get attached to the wrapper as attributes.  
Each of these accessors allows internal parameters to be adjusted through the use of nonlocal assignments.  
An added feature of this recipe is that the accessor functions will propagate through multiple levels of decoration if all of your decorators use `@functools.wraps`.  
For example, suppose you introduced an additional decorator, such as the `@timethis` decorator from Recipe 9.2, and wrote code like this:

In [26]:
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1

In [27]:
countdown(10000)

DEBUG:__main__:countdown


countdown 0.001477956771850586


In [28]:
countdown.set_level(logging.WARNING)
countdown.set_message("Ground control to Major Tom, prepare for blastoff")
countdown(10000)

countdown 0.0015249252319335938


If the decorators are composed in the exact opposite order, everything still works exactly the same:

In [29]:
@logged(logging.DEBUG)
@timethis
def countdown(n):
    while n > 0:
        n -= 1

You can make additional modifications, such as writing accessor functions to return the values of varous settings:

Or an alternative:

One extremely subtle facet of this recipe is the choice to use accessor functions in the first place.  
For example, you might consider an alternative formulation solely based on direct access to function attributes like this:

This approach would work to a point, but only if it was the topmost decorator.  
If you had another decorator applied on top, such as the `@timethis` example, it would shadow the underlying attributes and make them unavailable for modification.  
The use of accessor functions avoids this limitation.  
The solution shown in this recipe might also be a possible alternative for decorators defined as classes, as shown in Recipe 9.9.

## 9.6. Defining a Decorator That Takes an Optional Argument

### Problem

You would like to write a single decorator that can be used without arguments, such as `@decorator`, or with optional arguments, such as `@decorator(x,y,z)`.  
However, there seems to be no straightforward way to do it due to differences in calling conventions between simple decorators and decorators taking arguments.

### Solution

Here is a variant of the logging code shown in Recipe 9.5 that defines a decorator according to our specifications:

In [30]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)
    
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

Now for a demonstration:

In [31]:
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name = 'demonstration')
def spam():
    print('Spam Ahoy!')

As you can see from the example, the decorator can be used in both a simple form such as `@logged`, or with optional arguments supplied, like with `@logged(level=logging.CRITICAL, name='example')`.

### Discussion

The problem addressed by this recipe is really one of programming consistency.  
When using decorators, most programmers are used to applying them without any arguments at all or with arguments, as shown in the example.  
Technically speaking, a decorator where all arguments are optional could be applied, like this:

In [32]:
@logged()
def add(x, y):
    return x + y

However, this is not a form that’s especially common, and might lead to common usage errors if programmers forget to add the extra parentheses.  
The recipe simply makes the decorator work with or without parentheses in a consistent way.  
To understand how the code works, you need to have a firm understanding of how decorators get applied to functions as well as their calling conventions.  
For a simple decorator such as this:

In [33]:
@logged
def add(x, y):
    return x + y

The calling sequence is as follows:

In [34]:
def add(x, y):
    return x + y
add = logged(add)

In this case, the function to be wrapped is simply passed to logged as the first argument.  
Thus, in the solution, the first argument of `logged()` is the function being wrapped.  
All of the other arguments must have default values.  
For a decorator taking arguments such as this:

In [35]:
@logged(level=logging.CRITICAL, name='demonstration')
def spam():
    print('Spam Alert!')

The calling sequence goes:

In [36]:
def spam():
    print('Spam Alert!')
spam = logged(level=logging.CRITICAL, name ='demonstration')(spam)

On the initial invocation of `logged()`, the function to be wrapped is not passed.  
Thus, in the decorator, it has to be optional.  
This, in turn, forces the other arguments to be specified by keyword.  
Furthermore, when arguments are passed, a decorator is supposed to return a function that accepts the function and wraps it (see Recipe 9.5).  
To do this, the solution uses a clever trick involving `functools.partial`.  
Specifically, it simply returns a partially applied version of itself where all arguments are fixed except for the function to be wrapped.  
See Recipe 7.8 for more details about using `partial()`.

## 9.7. Enforcing Type Checking on a Function Using a Decorator

### Problem

You want to optionally enforce type checking of function arguments as a kind of assertion or contract.

### Solution

Before showing the solution code, the aim of this recipe is to have a means of enforcing type contracts on the input arguments to a function.  
Here is a short example that illustrates the idea:

Here is an implementation of the `@typeassert` decorator:

In [37]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func
        
        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                        )
            return func(*args, **kwargs)
        return wrapper
    return decorate

You will find that this decorator is rather flexible, allowing types to be specified for all or a subset of a function’s arguments.  
Moreover, types can be specified by position or by keyword.

In [38]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

In [39]:
spam(1, 2, 3)

1 2 3


In [40]:
spam(1, 'hello', 3)

1 hello 3


### Discussion

This recipe is an advanced decorator example that introduces a number of important and useful concepts.  
First, one aspect of decorators is that they only get applied once, at the time of function definition.  
In certain cases, you may want to disable the functionality added by a decorator.  
To do this, simply have your decorator function return the function unwrapped.  
In the solution, the following code fragment returns the function unmodified if the value of the global `__debug__` variable is set to `False`, which is the case when Python executes in optimized mode with the `-O` or `-OO` options to the interpreter.

In [41]:
# ...
def decorate(func):
    # If in optimized mode, disable type checking
    if not __debug__:
        return func
    # ...

Next, a tricky part of writing this decorator is that it involves examining and working with the argument signature of the function being wrapped.  
Your tool of choice here should be the `inspect.signature()` function.  
Simply stated, it allows you to extract signature information from a callable.

In [42]:
from inspect import signature
def spam(x, y, z=42):
    pass

In [43]:
sig = signature(spam)
print(sig)

(x, y, z=42)


In [44]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [45]:
sig.parameters['z'].name

'z'

In [46]:
sig.parameters['z'].default

42

In [47]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In the first part of our decorator, we use the `bind_partial()` method of signatures to perform a partial binding of the supplied types to argument names.  
Here is an example of what happens:

In [48]:
bound_types = sig.bind_partial(int, z=int)
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [49]:
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In this partial binding, you will notice that missing arguments are simply ignored, which is the case with argument y.  
However, the most important part of the binding is the creation of the ordered dictionary `bound_types.arguments`.  
This dictionary maps the argument names to the supplied values in the same order as the function signature.  
In the case of our decorator, this mapping contains the type assertions that we’re going to enforce.  
In the actual wrapper function made by the decorator, the `sig.bind()` method is used.  
The `bind()` method is like `bind_partial()` except that it does not allow for missing arguments.  
So, here is what happens:

In [50]:
bound_values = sig.bind(1, 2, 3)
bound_values.arguments

OrderedDict([('x', 1), ('y', 2), ('z', 3)])

Using this mapping, enforcing the required assertions is pretty straightforward:

In [51]:
for name, value in bound_values.arguments.items():
    if name in bound_types.arguments:
        if not isinstance(value, bound_types.arguments[name]):
            raise TypeError()

A somewhat subtle aspect of the solution is that the assertions do not get applied to unsupplied arguments with default values.  
For example, this code works, even though the default value of items is of the "wrong" type:

In [52]:
@typeassert(int, list)
def bar(x, items = None):
    if items is None:
        items = []
    items.append(x)
    return items

In [53]:
bar(2)

[2]

In [54]:
bar(4, [1, 2, 3])

[1, 2, 3, 4]

A final point of design discussion might be the use of decorator arguments versus function annotations.  
For example, why not write the decorator to look at annotations like this?

One possible reason for not using annotations is that each argument to a function can only have a single annotation assigned.  
Thus, if the annotations are used for type assertions, they can’t really be used for anything else.  
Likewise, the `@typeassert` decorator won’t work with functions that use annotations for a different purpose.  
By using decorator arguments, as shown in the solution, the decorator becomes a lot more general purpose and can be used with any function whatsoever -- even functions that use annotations.  
More information about function signature objects can be found in PEP 362, as well as the documentation for the `inspect` module.  
Recipe 9.16 also has an additional example.

## 9.8. Defining Decorators As Part of a Class

### Problem

You want to define a decorator inside a class definition and apply it to other functions or methods.

### Solution

Before you define a decorator inside a class, you need to decide whether it is applied as an instance or a class method.  
Let's take a look at the difference:

In [55]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1 as an instance method.')
            return func(*args, **kwargs)
        return wrapper
    
    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2 as a class method.')
            return func(*args, **kwargs)
        return wrapper

Here is how we can apply our decorators:

In [56]:
# As an instance method
a = A()

@a.decorator1
def spam():
    pass

In [57]:
# As a class method
@A.decorator2
def foo():
    pass

Notice that one is applied from instance `a` and the other is applied from class `A`.

### Discussion

Defining decorators in a class might look odd at first glance, but there are examples of this in the standard library.  
In particular, the built-in `@property` decorator is actually a class with `getter()`, `setter()`, and `deleter()` methods that each act as a decorator.

In [58]:
class Person:
    # Create a property instance
    first_name = property()
    
    # Apply decorator methods
    @first_name.getter
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Must be a string')
        self._first_name = value

The key reason why it’s defined in this way is that the various decorator methods are manipulating state on the associated property instance.  
So, if you ever had a problem where decorators needed to record or combine information behind the scenes, it’s a sensible approach.  
A common confusion when writing decorators in classes is getting tripped up by the proper use of the extra `self` or `cls` arguments in the decorator code itself.  
Although the outermost decorator function, such as `decorator1()` or `decorator2()`, needs to provide a `self` or `cls` argument (since they’re part of a class), the wrapper function created inside doesn’t generally need to include an extra argument.  
This is why the `wrapper()` function created in both decorators doesn’t include a `self` argument.  
The only time you would ever need this argument is in situations where you actually needed to access parts of an instance in the wrapper.  
Otherwise, you just don’t have to worry about it.  
A final subtle facet of having decorators defined in a class concerns their potential use with inheritance.  
For example, suppose you want to apply one of the decorators defined in class `A` to methods defined in a subclass `B`.  
To do that, you would need to write code like this:

In [59]:
class B(A):
    @A.decorator2
    def bar(self):
        pass

In particular, the decorator in question has to be defined as a class method and you have to explicitly use the name of the superclass `A` when applying it.  
You can’t use a name such as @`B.decorator2`, because at the time of method definition, class `B` has not yet been created.

## 9.9 Defining Decorators As Classes

### Problem

You want to wrap functions with a decorator, but the result is going to be a callable instance.  
You need your decorator to work both inside and outside class definitions.

### Solution

To define a decorator as an instance, you need to make sure it implements the `__call__()` and `__get__()` methods.  
For example, this code defines a class that puts a simple profiling layer around another function:

In [60]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

You can use this class like a decorator either inside or outside of another class:

In [61]:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

In [62]:
add(2, 3)

5

In [63]:
add(4, 5)

9

In [64]:
add.ncalls

2

In [65]:
s = Spam()
s.bar(1)

<__main__.Spam object at 0x106711be0> 1


In [66]:
s.bar(2)

<__main__.Spam object at 0x106711be0> 2


In [67]:
s.bar(3)

<__main__.Spam object at 0x106711be0> 3


In [68]:
Spam.bar.ncalls

3

### Discussion

Defining a decorator as a class is usually straightforward.  
However, there are some rather subtle details that deserve more explanation, especially if you plan to apply the decorator to instance methods.  
First, the use of the `functools.wraps()` function serves the same purpose here as it does in normal decorators -- namely to copy important metadata from the wrapped function to the callable instance.  
Second, it is common to overlook the `__get__()` method shown in the solution.  
If you omit the `__get__()` and keep all of the other code the same, you’ll find that bizarre things happen when you try to invoke decorated instance methods.  
The reason it breaks is that whenever functions implementing methods are looked up in a class, their `__get__()` method is invoked as part of the descriptor protocol, which is described in Recipe 8.9.  
In this case, the purpose of `__get__()` is to create a bound method object which ultimately supplies the self argument to the method.

In [69]:
s = Spam()
def grok(self, x):
    pass

In [70]:
grok.__get__(s, Spam)

<bound method grok of <__main__.Spam object at 0x106732358>>

In this recipe, the `__get__()` method is there to make sure bound method objects get created properly.  `type.MethodType()` creates a bound method manually for use here.  
Bound methods only get created if an instance is being used.  
If the method is accessed on a class, the instance argument to `__get__()` is set to `None` and the `Profiled` instance itself is just returned.  
This makes it possible for someone to extract its `ncalls` attribute, as shown.  
If you want to avoid some of this of this mess, you might consider an alternative formulation of the decorator using closures and nonlocal variables, as described in Recipe 9.5.

In [71]:
import types
from functools import wraps

def profiled(func):
    ncalls =0
    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal ncalls
        ncalls += 1
        return func(*args, **kwargs)
    wrapper.ncalls = lambda: ncalls
    return wrapper

# Example
@profiled
def add(x, y):
    return x + y

Now access to `ncalls` is provided through a function attached as a function attribute.

In [72]:
add(2, 3)

5

In [73]:
add(4, 5)

9

In [74]:
add.ncalls()

2

## 9.10. Applying Decorators to Class and Static Methods

### Problem

You want to apply a decorator to a class or static method.

### Solution

Applying decorators to class and static methods is straightforward, but make sure that your decorators are applied before `@classmethod` or `@staticmethod`.

In [75]:
import time
from functools import wraps

# A simple timing function:
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper

Now we can write a class illustrating the application of this decorator to different kinds of methods.

In [76]:
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1
            
    @classmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

The resulting class and static methods should operate normally, but have the extra timing.

In [77]:
s = Spam()
s.instance_method(100000)

<__main__.Spam object at 0x106732e10> 100000
0.006449222564697266


### Discussion

If you don't properly order the decorators you will probably get an error.

In [78]:
class Spam:
    # ...
    @timethis
    @staticmethod
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

The problem here is that `@classmethod` and `@staticmethod` don’t actually create objects that are directly callable.  
Instead, they create special descriptor objects, as described in Recipe 8.9.  
Thus, if you try to use them like functions in another decorator, the decorator will crash.  
Making sure that these decorators appear first in the decorator list fixes the problem.  
One situation where this recipe is of critical importance is in defining class and static methods in abstract base classes, as described in Recipe 8.12.  
For example, if you want to define an abstract class method, you can use this code:

In [79]:
from abc import ABCMeta, abstractmethod

class A(metaclass=ABCMeta):
    @classmethod
    @abstractmethod
    def method(cls):
        pass

In this code, the order of `@classmethod` and `@abstractmethod` matters.  
If you flip the two decorators around, everything breaks.

## 9.11. Writing Decorators That Add Arguments to Wrapped Functions

### Problem

You want to write a decorator that adds an extra argument to the calling signature of the wrapped function.  
However, the added argument can’t interfere with the existing calling conventions of the function.  

### Solution

Extra arguments can be injected into the calling signature using keyword-only arguments.  
Consider the following decorator:  

In [80]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper

Here is an example of how the decorator works:

In [81]:
@optional_debug
def spam(a, b, c):
    print(a, b, c)

In [82]:
spam(1,2,3)

1 2 3


In [83]:
spam(1,2,3, debug=True)

Calling spam
1 2 3


### Discussion

Adding arguments to the signature of wrapped functions is not the most common example of using decorators.  
However, it might be a useful technique in avoiding certain kinds of code replication patterns.  

In [84]:
def a(x, debug=False):
    if debug:
        print('Calling a')
    # ...
    
def b(x, y, z, debug=False):
    if debug:
        print('Calling b')
    # ...
    
def c(x, y, debug=False):
    if debug:
        print('Calling c')

You can now refactor:

In [85]:
@optional_debug
def a(x):
    """
    ...
    """
    
@optional_debug
def b(x, y, z):
    """
    ...
    """
    
@optional_debug
def c(x, y):
    """
    @optional_debug
    """

The implementation of this recipe relies on the fact that keyword-only arguments are easy to add to functions that also accept `*args` and `**kwargs` parameters.  
By using a keyword-only argument, it gets singled out as a special case and removed from subsequent calls that only use the remaining positional and keyword arguments.  
One tricky part here concerns a potential name clash between the added argument and the arguments of the function being wrapped.  
For example, if the `@optional_debug` decorator was applied to a function that already had a debug argument, then it would break.  
If that’s a concern, an extra check could be added:

In [86]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.getfullargspec(func).args:
        raise TypeError('debug argument already defined')
        
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper

A final refinement to this recipe concerns the proper management of function signatures.  
An astute programmer will realize that the signature of wrapped functions is wrong.

In [87]:
@optional_debug
def add(x, y):
    return x + y

import inspect
print(inspect.signature(add))

(x, y)


Great, how do we fix the problem?

In [88]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.getfullargspec(func).args:
        raise TypeError('debug argument already defined')
        
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    
    sig = inspect.signature(func)
    parms = list((sig.parameters.values()))
    parms.append(inspect.Parameter('debug',
                                   inspect.Parameter.KEYWORD_ONLY,
                                   default=False))
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

With this change, the signature of the wrapper will now correctly reflect the presence of the `debug` argument.

In [89]:
@optional_debug
def add(x,y):
    return x + y

print(inspect.signature(add))

(x, y, *, debug=False)


In [90]:
add(2,3)

5

See Recipe 9.16 for more information about function signatures.

## 9.12. Using Decorators to Patch Class Definitions

### Problem


You want to inspect or rewrite portions of a class definition to alter its behavior, but without using inheritance or metaclasses.

### Solution

This is a perfect use for a class decorator.  
Here is a class decorator that rewrites the `__getattribute__` special method to perform logging.

In [91]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__
    
    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)
    
    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

In [92]:
# Example use:
@log_getattribute
class A:
    def __init__(self, x):
        self.x = x
    def spam(self):
        pass

Let's use this class in the solution:

In [93]:
a = A(42)
a.x

getting: x


42

In [94]:
a.spam()

getting: spam


### Discussion

Class decorators can often be used as a straightforward alternative to other more advanced techniques involving mixins or metaclasses.  
For example, an alternative implementation of the solution might involve inheritance, as in the following:

In [95]:
class LoggedGetattribute:
    def __getattribute__(self, name):
        print('getting:', name)
        return super().__getattribute__(name)

In [96]:
# Example:
class A(LoggedGetattribute):
    def __init__(self, x):
        self.x = x
    def spam(self):
        pass

This works, but to understand it, you have to have some awareness of the method resolution order, `super()`, and other aspects of inheritance, as described in Recipe 8.7.  
In some sense, the class decorator solution is much more direct in how it operates, and it doesn’t introduce new dependencies into the inheritance hierarchy.  
As it turns out, it’s also just a bit faster, due to not relying on the `super()` function.  
If you are applying multiple class decorators to a class, the application order might matter.  
For example, a decorator that replaces a method with an entirely new implementation would probably need to be applied before a decorator that simply wraps an existing method with some extra logic.  
See Recipe 8.13 for another example of class decorators in action.

## 9.13. Using a Metaclass to Control Instance Creation

### Problem

You want to change the way in which instances are created in order to implement singletons, caching, or other similar features.

### Solution

With Python, once you define a class, you call it like a function to create instances.

In [97]:
class Spam:
    def __init__(self, name):
        self.name = name

In [98]:
a = Spam('Alpha')
b = Spam('Beta')

If you want to customize this step, you can do it by defining a metaclass and reimplementing its `__call__()` method in some way.  
To illustrate, suppose that you didn’t want anyone creating instances at all:

In [99]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("No direct instantiation")
        
# Example
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In this case, users can call the defined static method, but it’s impossible to create an instance in the normal way.

In [100]:
Spam.grok(42)

Spam.grok


Now, suppose you want to implement the singleton pattern, which is a class where only one instance is ever created:

In [101]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
        
    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance

Now an example:

In [102]:
class Spam(metaclass=Singleton):
    def __init__(self):
        print("Creating Spam")

In this case, only one instance is ever created.

In [103]:
a = Spam()

Creating Spam


In [104]:
b = Spam()
a is b

True

In [105]:
c = Spam()
a is c

True

In [106]:
b is c

True

Finally, suppose you want to create cached instances, as described in Recipe 8.25.  
Here’s a metaclass that implements it:

In [107]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
        
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

In [108]:
# Example
class Spam(metaclass=Cached):
    def __init__(self, name):
        print("Creating Spam({!r})".format(name))
        self.name = name

In [109]:
a = Spam('Alpha')

Creating Spam('Alpha')


In [110]:
b = Spam('Beta')

Creating Spam('Beta')


In [111]:
c = Spam('Alpha')  # Cached

In [112]:
a is b

False

In [113]:
a is c  # Cached value returned

True

### Discussion

Using a metaclass to implement various instance creation patterns can often be a much more elegant approach than other solutions not involving metaclasses.  
For example, if you didn’t use a metaclass, you might have to hide the classes behind some kind of extra factory function.  
For example, to get a singleton, you might use a hack such as the following:

In [114]:
class _Spam:
    def __init__(self):
        print('Creating Spam')
        
_spam_instance = None
def Spam():
    global _spam_instance
    if _spam_instance is not None:
        return _spam_instance
    else:
        _spam_instance = _Spam()
        return _spam_instance

Although the solution involving metaclasses involves a much more advanced concept, the resulting code feels cleaner and less hacked together.  
See Recipe 8.25 for more information on creating cached instances, weak references, and other details.

## 9.14. Capturing Class Attribute Definition Order

### Problem

You want to automatically record the order in which attributes and methods are defined inside a class body so that you can use it in various operations such as serializing and mapping to databases.

### Solution

Capturing information about the body of class definition is easily accomplished through the use of a metaclass.  
Here is an example of a metaclass that uses an `OrderedDict` to capture definition order of descriptors:

In [115]:
from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value
        
class Integer(Typed):
    _expected_type = int
    
class Float(Typed):
    _expected_type = float
    
class String(Typed):
    _expected_type = str
    
# Metaclass that uses an OrderedDict for the class body
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

@classmethod
def __prepare__(cls, clsname, bases):
    return OrderedDict()

In this metaclass, the definition order of descriptors is captured by using an `Ordered Dict` during the execution of the class body.  
The resulting order of names is then extracted from the dictionary and stored into a class attribute `_order`.  
This can then be used by methods of the class in various ways.  
For example, here is a simple class that uses the ordering to implement a method for serializing the instance data as a line of CSV data:

In [116]:
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)
    
# Example use
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [117]:
s = Stock('GOOG',100,490.1)
s.name

'GOOG'

In [118]:
s.as_csv()

'GOOG,100,490.1'

### Discussion

The key to this recipe is the `__prepare__()` method, which is defined in the `OrderedMeta` metaclass.  
This method is invoked immediately at the start of a class definition with the class name and base classes.  
It must then return a mapping object to use when processing the class body.  
By returning an `OrderedDict` instead of a normal dictionary, the resulting definition order is easily captured.  
It is possible to extend this functionality even further if you are willing to make your own dictionary-like objects. For example, consider this variant of the solution that rejects duplicate definitions:

In [119]:
class NoDupOrderedDict(OrderedDict):
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()
    def __setitem__(self, name, value):
        if name in self:
            raise TypeError('{} already defined in {}'.format(name, self.clsname))
        super().__setitem__(name, value)
        
class OrederdMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_']
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return NoDupOrderedDict(clsname)

Now see what happens when you use this class and make a class with duplicate entries:

In [120]:
class A(metaclass=OrderedMeta):
    def spam(self):
        pass
    def spam(self):
        pass

An important part this recipe concerns the treatment of the modified dictionary in the metaclass `__new__()` method.  
Even though the class was defined using an alternative dictionary, you still have to convert this dictionary to a proper `dict` instance when making the final class object.  
This is the purpose of the `d = dict(clsdict)` statement.  
Being able to capture definition order is a subtle but important feature for certain kinds of applications.  
For instance, in an object relational mapper, classes might be written in a manner similar to that shown in the example:

Underneath the covers, the code might want to capture the definition order to map objects to tuples or rows in a database table, similar to the functionality of the `as_csv()` method in the example.  
The solution shown is very straightforward and often simpler than alternative approaches which typically involve maintaining hidden counters within the descriptor classes.

## 9.15. Defining a Metaclass That Takes Optional Arguments

### Problem

You want to define a metaclass that allows class definitions to supply optional arguments, possibly to control or configure aspects of processing during type creation.

### Solution

When defining classes, Python allows a metaclass to be specified using the `metaclass` keyword argument in the class statement.  
For example, with abstract base classes:

In [121]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxsize=None):
        pass
    
    @abstractmethod
    def write(self, data):
        pass

However, in custom metaclasses, additional keyword arguments can be supplied:

To support such keyword arguments in a metaclass, make sure you define them on the `__prepare__()`, `__new__()`, and `__init__()` methods using keyword-only arguments, like this:

In [122]:
class MyMeta(type):
    # Optional
    @classmethod
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):
        """
        Custom processing takes place here
        """
        # ...
        return super().__prepare__(name, bases)
    
    # Required
    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):
        """
        Custom processing
        """
        # ...
        return super().__new__(cls, name, bases, ns)
    
    # Required
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        """
        More custom processing
        """
        super().__init__(name, bases, ns)

### Discussion

Adding optional keyword arguments to a metaclass requires that you understand all of the steps involved in class creation, because the extra arguments are passed to every method involved.  
The `__prepare__()` method is called first and used to create the class namespace prior to the body of any class definition being processed.  
Normally, this method simply returns a dictionary or other mapping object.  
The `__new__()` method is used to instantiate the resulting type object.  
It is called after the class body has been fully executed.  
The `__init__()` method is called last and used to perform any additional initialization steps.

When writing metaclasses, it is somewhat common to only define a `__new__()` or `__init__()` method, but not both.  
However, if extra keyword arguments are going to be accepted, then both methods must be provided and given compatible signatures.  
The default `__prepare__()` method accepts any set of keyword arguments, but ignores them.  
You only need to define it yourself if the extra arguments would somehow affect management of the class namespace creation.  
The use of keyword-only arguments in this recipe reflects the fact that such arguments will only be supplied by keyword during class creation.  
The specification of keyword arguments to configure a metaclass might be viewed as an alternative to using class variables for a similar purpose.  
Examine the following code:

The advantage to supplying such parameters as an argument is that they don’t pollute the class namespace with extra names that only pertain to class creation and not the subsequent execution of statements in the class.  
In addition, they are available to the `__prepare__()` method, which runs prior to processing any statements in the class body.  
Class variables, on the other hand, would only be accessible in the `__new__()` and `__init__()` methods of a metaclass.

## 9.16. Enforcing an Argument Signature on `*args` and `**kwargs`

### Problem

You’ve written a function or method that uses `*args` and `**kwargs`, so that it can be general purpose, but you would also like to check the passed arguments to see if they match a specific function calling signature.

### Solution

For any problem where you want to manipulate function calling signatures, you should use the signature features found in the inspect module.  
Two classes, `Signature` and `Parameter`, are of particular interest here.  
Here is an interactive example of creating a function signature:

In [123]:
from inspect import Signature, Parameter

# Make a signature for a func(x, y=42, *, z=None)
parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
          Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
          Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
sig = Signature(parms)
print(sig)

(x, y=42, *, z=None)


Once you have a signature object, you can easily bind it to `*args` and `**kwargs` using the signature's `bind()` method.

In [124]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name, value)

In [125]:
# Examples:
func(1, 2, z=3)

x 1
y 2
z 3


In [126]:
func(1)

x 1


In [127]:
func(1, z=3)

x 1
z 3


In [128]:
func(y=2, x=1)

x 1
y 2


As you can see, the binding of a signature to the passed arguments enforces all of the usual function calling rules concerning required arguments, defaults, duplicates, and so forth.  
Here is a more concrete example of enforcing function signatures. In this code, a base class has defined an extremely general-purpose version of `__init__()`, but subclasses are expected to supply an expected signature.

In [129]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
             for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

Example use:

In [130]:
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')
    
class Point(Structure):
    __signature__ = make_sig('x', 'y')

In [131]:
import inspect
print(inspect.signature(Stock))

(name, shares, price)


In [132]:
s1 = Stock('ACME', 100, 490.1)

### Discussion

The use of functions involving `*args` and `**kwargs` is very common when trying to make general-purpose libraries, write decorators or implement proxies.  
However, one downside of such functions is that if you want to implement your own argument checking, it can quickly become an unwieldy mess.  
As an example, see Recipe 8.11.  
The use of a signature object simplifies this.  
In the last example of the solution, it might make sense to create signature objects through the use of a custom metaclass.  
Here is an alternative implementation that shows how to do this:

In [133]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
            for name in names]
    return Signature(parms)

class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[]))
        return super().__new__(cls, clsname, bases, clsdict)
    
class Structure(metaclass=StructureMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

In [134]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
class Point(Structure):
    _fields = ['x', 'y']

When defining custom signatures, it is often useful to store the signature in a special attribute `__signature__`, as shown.  
If you do this, code that uses the inspect module to perform introspection will see the signature and report it as the calling convention.

In [135]:
import inspect

print(inspect.signature(Stock))

(name, shares, price)


In [136]:
print(inspect.signature(Point))

(x, y)


## 9.17. Enforcing Coding Conventions in Classes